In [2]:
#!pip install --upgrade gensim
import gensim
import json
import jsonpickle
from flask import Flask, request

import config
from gensim.summarization.bm25 import get_bm25_weights, BM25
import re

In [3]:
app = Flask(__name__)

@app.route('/ranking', methods=["POST"])
def ranging():
    documents = jsonpickle.decode(request.json)
    corpus = []
    for doc in documents["documents"]:

        text = re.split(" ", doc.text_normalized)
        corpus.append(text)
    
    query = re.split(" ", documents["query"])
    corpus.append(query)
    #print(corpus)
    
    bm25 = BM25(corpus)
    average_idf = sum(map(lambda k: float(bm25.idf[k]), bm25.idf.keys())) / len(bm25.idf.keys())
    scores = bm25.get_scores(query, average_idf)
    
    sorted_corp = list(zip(scores[:-1], documents["documents"]))
    ranked_docs = sorted(sorted_corp, key=lambda kv: kv[0], reverse=True)
    
    return jsonpickle.encode(ranked_docs)


In [ ]:
if __name__ == "__main__":
    app.run(host=config.RANKING_HOST, port=config.RANKING_PORT)


 * Running on http://127.0.0.1:13502/ (Press CTRL+C to quit)
127.0.0.1 - - [29/Nov/2018 16:25:50] "POST /ranking HTTP/1.1" 200 -
127.0.0.1 - - [29/Nov/2018 16:27:55] "POST /ranking HTTP/1.1" 200 -
127.0.0.1 - - [29/Nov/2018 16:28:31] "POST /ranking HTTP/1.1" 200 -
127.0.0.1 - - [29/Nov/2018 16:28:51] "POST /ranking HTTP/1.1" 200 -
